# Predictive Modeling of Hospital Length of Stay and Discharge Type
# [Step 3: Merge laboratory and clinical data]

## 1. Import libraries and load datasets

In [16]:
# Import data manipulation library
import pandas as pd
import numpy as np

In [2]:
# Define file paths for cleaned input datasets  
lab_data_path = "/home/anna/Desktop/Master_thesis/output_data/cleaned_lab_data" 
clinical_data_path = "/home/anna/Desktop/Master_thesis/output_data/cleaned_clinical_data"  

# Define output path for merged dataset  
merged_data_path = "/home/anna/Desktop/Master_thesis/output_data/merged_data"  

In [3]:
# Load lab dataset 
lab_data = pd.read_csv(lab_data_path)

In [4]:
display(lab_data)

,patient_id,case_id,test_name,test_abbr,method_number,numeric_result,text_result,unit
0,1,171465,Natrium,Na,1,138.0,138,mmol/L
1,1,171465,Kalium,KA,3,4.6,4.6,mmol/L
2,1,171465,Hämolytisch,H-Se,42,4.0,4,Unknown
3,1,171465,Lipämisch,L-Se,43,3.0,3,Unknown
4,1,171465,Ikterisch,I-Se,44,1.0,1,Unknown
...,...,...,...,...,...,...,...,...
19313528,240990,415184,MCHC,MCHCn,9116,333.0,333,g/L
19313529,240990,415184,RDW,RDWn,9117,14.3,14.3,%
19313530,240990,415184,Thrombozyten,THZn,9119,226.0,226,G/L
19313531,240990,415184,MPV,MPVn,9204,9.0,9.0,fL


In [5]:
# Load and display clinical dataset
clinical_data = pd.read_csv(clinical_data_path)
display(clinical_data)

,patient_id,case_id,discharge_type,sex,age,length_of_stay_days,diagnosis,diagnosis_category
0,1,171465,Entlassung,f,73,28,A04.70,Infectious diseases
1,1,333396,Entlassung,f,73,34,K57.22,Digestive diseases
2,2,27091,Entlassung,m,51,13,T84.5,Injury & poisoning
3,2,36154,Entl.ext.Instit,m,50,12,I21.4,Circulatory diseases
4,2,142617,Entl. in ex.KH,m,51,1,T84.5,Injury & poisoning
...,...,...,...,...,...,...,...,...
273558,240988,393440,Entlassung,m,79,2,K55.21,Digestive diseases
273559,240988,412516,Entlassung,m,79,10,I11.00,Circulatory diseases
273560,240988,425806,Entl.ext.Instit,m,79,7,M16.7,Musculoskeletal diseases
273561,240989,393141,Entlassung,m,62,1,I49.3,Circulatory diseases


In [6]:
# Display results
print(lab_data.nunique())

patient_id        182214
case_id           311547
test_name           2893
test_abbr           3891
method_number       3978
numeric_result     62295
text_result        98346
unit                 167
dtype: int64


## 2. Filter out lab tests with more than 80% missing data  

In [7]:
# Select most frequent lab tests based on case occurrence

# Count unique cases per lab test
lab_test_counts = lab_data.groupby(['test_abbr', 'test_name', 'unit'], as_index=False)['case_id'].nunique()
lab_test_counts.rename(columns={'case_id': 'num_cases'}, inplace=True)

# Get total number of unique cases
total_cases = lab_data["case_id"].nunique()

# Calculate missing percentage as cases that do NOT have the test
lab_test_counts["missing_percentage"] = ((total_cases - lab_test_counts["num_cases"]) / total_cases) * 100

# Sort lab tests by frequency (most frequent first)
lab_test_counts.sort_values(by="num_cases", ascending=False, inplace=True)

# Filter lab tests that have less than 80% missing data
filtered_lab_tests = lab_test_counts[lab_test_counts["missing_percentage"] < 80][["test_abbr", 'test_name', 'unit', "missing_percentage"]]


In [8]:
pd.set_option('display.max_rows', None)  # Show all rows
display(filtered_lab_tests)

,test_abbr,test_name,unit,missing_percentage
1913,KA,Kalium,mmol/L,14.160945
2230,Leukn,Leukozyten,G/L,15.383875
1782,Hbn,Hämoglobin,g/L,15.384838
1308,Eryn,Erythrozyten,T/L,15.386442
1799,Hkn,Hämatokrit,L/L,15.387405
3306,THZn,Thrombozyten,G/L,15.389010
2290,MCHCn,MCHC,g/L,15.389331
2296,MCVn,MCV,fL,15.389652
2292,MCHn,MCH,pg,15.389652
2925,RDWn,RDW,%,15.462194


### 2.1 Removal of Specific Lab Tests  

Excluded tests fall into two categories:  

1. **Metadata Entries** (non-result data)  
2. **Highly Variable Urinary Tests** (fluctuate reducing reliability)  

In [12]:
remove_tests = [
    "EC3-U", "PH4-U", "BI3-U", "LK3-U", "PROT3", "NITR3", "KETO3", "SPEZ3",
    "FARBE3", "TRUEB3", "UST1", "BA-Ux", "LK-Ux", "KRI-Ux", "ERY-Ux", "PI-Ux", "EART",
    "GLUC3", "URO3", "ENTN1n", "Ben-ID", "TNT_hn"
]

# Remove specified lab tests from the filtered list  
filtered_lab_tests = filtered_lab_tests[~filtered_lab_tests["test_abbr"].isin(remove_tests)]

In [ ]:
display(filtered_lab_tests)

# Print the number of rows (tests) in filtered_lab_tests
print(f"Number of tests in filtered_lab_tests: {filtered_lab_tests.shape[0]}")

,test_abbr,test_name,unit,missing_percentage
1913,KA,Kalium,mmol/L,14.160945
2230,Leukn,Leukozyten,G/L,15.383875
1782,Hbn,Hämoglobin,g/L,15.384838
1308,Eryn,Erythrozyten,T/L,15.386442
1799,Hkn,Hämatokrit,L/L,15.387405
3306,THZn,Thrombozyten,G/L,15.389010
2290,MCHCn,MCHC,g/L,15.389331
2296,MCVn,MCV,fL,15.389652
2292,MCHn,MCH,pg,15.389652
2925,RDWn,RDW,%,15.462194


Number of tests in filtered_lab_tests: 49


In [11]:
# Extract the relevant columns (test_abbr, test_name, and unit) from the dataset 
# and save them as a new table for later use.
reference_table = filtered_lab_tests[['test_abbr', 'test_name', 'unit']]
reference_table.to_csv('lab_test_reference_table.csv', index=False)

In [17]:
# Filter the lab_data to keep only rows where test_abbr, test_name, and unit match the filtered list
lab_data_filtered = lab_data.merge(filtered_lab_tests, on=["test_abbr", "test_name", "unit"], how="inner")

# Display the first few rows of the filtered data
lab_data_filtered.head()

,patient_id,case_id,test_name,test_abbr,method_number,numeric_result,text_result,unit,missing_percentage
0,1,171465,Natrium,Na,1,138.0,138,mmol/L,16.194346
1,1,171465,Kalium,KA,3,4.6,4.6,mmol/L,14.160945
2,1,171465,Hämolytisch,H-Se,42,4.0,4,Unknown,79.562955
3,1,171465,Lipämisch,L-Se,43,3.0,3,Unknown,79.562955
4,1,171465,Ikterisch,I-Se,44,1.0,1,Unknown,79.562955


In [ ]:
lab_data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10724655 entries, 0 to 10724654
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   patient_id          int64  
 1   case_id             int64  
 2   test_name           object 
 3   test_abbr           object 
 4   method_number       int64  
 5   numeric_result      float64
 6   text_result         object 
 7   unit                object 
 8   missing_percentage  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 736.4+ MB


In [28]:
# Free up memory
del lab_data

## 3. Merge the two data sets

In [18]:
# Convert lab data to wide format, keeping only the first test result per patient-case
lab_data_wide = lab_data_filtered.pivot_table(
    index=['patient_id', 'case_id'],  # Keep patient_id and case_id as index
    columns='test_abbr',  # Pivot on 'test_abbr'
    values='numeric_result',  # Use numeric_result as values
    aggfunc='first' # First measurement = Initial test at admission
).reset_index()

# Display the first few rows
lab_data_wide.head()

test_abbr,patient_id,case_id,ALAT,AP,ASAT,BASm#n,BIg,CA,CK,CO-HB,...,Quicks,RDWn,THZn,TNThsn,Tbga,UREA,pCO2,pH,pO2,tHb
0,1,171465,10.0,NaN,18.0,NaN,NaN,NaN,NaN,NaN,...,13.2,18.3,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,333396,19.0,93.0,22.0,0.19,8.0,2.02,NaN,NaN,...,11.6,18.8,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,27091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,18.9,164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,36154,38.0,58.0,NaN,NaN,NaN,NaN,347.0,NaN,...,12.7,14.4,131.0,367.0,NaN,4.5,NaN,NaN,NaN,NaN
4,2,142617,15.0,65.0,19.0,0.04,16.0,2.39,NaN,NaN,...,19.2,17.6,159.0,NaN,NaN,5.2,NaN,NaN,NaN,NaN


In [19]:
# Merge lab_data with clinical_data on patient_id and case_id
merged_data = pd.merge(clinical_data, lab_data_wide, on=['patient_id', 'case_id'], how='inner')

## 4. First clean of the merged dataset

### 4.1 Check for duplicates

In [20]:
# Check for duplicate rows
duplicates = merged_data.duplicated()

# Count total duplicate rows
num_duplicates = duplicates.sum()
print(f"Total duplicate rows: {num_duplicates}")

# Display some duplicate rows (if they exist)
if num_duplicates > 0:
    display(merged_data[duplicates].head())

Total duplicate rows: 0


### 4.2 Check for negative entries

In [21]:
pd.set_option('display.max_columns', None) # Diplay all the columns

# Count total negative values in columns from the 9th column onward
num_negative_entries = (merged_data.iloc[:, 8:] < 0).sum().sum()
print(f"Total number of negative entries: {num_negative_entries}")

#display(negative_rows)

Total number of negative entries: 494


In [22]:
# Select numeric columns from the 9th column onward and check for negative values
negative_columns = merged_data.iloc[:, 8:].columns[(merged_data.iloc[:, 8:] < 0).any()]
print("Columns with negative values:", list(negative_columns))

Columns with negative values: ['ALAT', 'ASAT', 'BIg', 'CA', 'CO-HB', 'CR', 'CRP', 'H', 'I', 'MTHB', 'QUHD']


In [23]:
# Replace negative values with NaN using
merged_data[list(negative_columns)] = merged_data[list(negative_columns)].apply(lambda col: col.map(lambda x: np.nan if x < 0 else x))

### 4.3 Post-Merge Filtering of Lab Tests with Excessive Missing Data (Above 80%) 

In [24]:
# Calculate the percentage of missing values for each column
missing_percentage = (merged_data.isnull().sum() / len(merged_data)) * 100

# Filter out columns with more than 80% missing data
merged_data = merged_data.loc[:, missing_percentage <= 80]

# Recalculate the missing percentage for the cleaned DataFrame
missing_percentage_cleaned = (merged_data.isnull().sum() / len(merged_data)) * 100

# Create a DataFrame to display the result
missing_percentage_df = pd.DataFrame({
    'Lab Test': merged_data.columns,
    'Missing Entries (%)': missing_percentage_cleaned
})

# Sort the DataFrame
missing_percentage_df = missing_percentage_df.sort_values(by='Missing Entries (%)', ascending=True).reset_index(drop=True)

# Set pandas option to display all rows
pd.set_option('display.max_rows', None)

# Display the DataFrame
display(missing_percentage_df)

,Lab Test,Missing Entries (%)
0,patient_id,0.000000
1,case_id,0.000000
2,discharge_type,0.000000
3,sex,0.000000
4,age,0.000000
5,length_of_stay_days,0.000000
6,diagnosis,0.000000
7,diagnosis_category,0.000000
8,KA,8.318668
9,Leukn,10.284209


## 5. Display and save merged dataset

In [39]:
merged_data.head()

,patient_id,case_id,discharge_type,sex,age,length_of_stay_days,diagnosis,diagnosis_category,ALAT,AP,ASAT,BASm#n,BIg,CA,CK,CR,CRP,EOSm#n,EPIGFR,Eryn,GGT,GL,H,H-Se,Hbn,Hkn,I,I-Se,IGm#n,INRiH,KA,L,L-Se,LACT,LYMm#n,Leukn,MCHCn,MCHn,MCVn,MONm#n,MPVn,NEUm#n,NRBCmn,Na,QUHD,Quicks,RDWn,THZn,TNThsn,UREA,tHb
0,1,171465,Entlassung,f,73,28,A04.70,Infectious diseases,10.0,NaN,18.0,NaN,NaN,NaN,NaN,57.0,22.0,NaN,89.0,4.70,51.0,6.00,10.0,4.0,98.0,0.31,1.0,1.0,NaN,1.28,4.6,6.0,3.0,NaN,NaN,21.30,320.0,21.0,65.0,NaN,10.0,NaN,0.0,138.0,57.8,13.2,18.3,165.0,NaN,NaN,NaN
1,1,333396,Entlassung,f,73,34,K57.22,Digestive diseases,19.0,93.0,22.0,0.19,8.0,2.02,NaN,NaN,34.0,0.03,NaN,4.76,59.0,5.50,2.0,NaN,104.0,0.34,1.0,NaN,3.30,1.12,4.0,13.0,NaN,1.9,1.89,30.90,323.0,24.0,75.0,6.73,8.1,18.85,0.0,137.0,77.0,11.6,18.8,93.0,NaN,NaN,NaN
2,2,27091,Entlassung,m,51,13,T84.5,Injury & poisoning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,230.0,NaN,103.0,4.73,NaN,5.75,6.0,NaN,128.0,0.38,1.0,NaN,NaN,1.91,3.9,3.0,NaN,NaN,NaN,7.12,328.0,26.0,80.0,NaN,11.6,NaN,0.0,136.0,55.8,14.0,18.9,164.0,NaN,NaN,NaN
3,2,36154,Entl.ext.Instit,m,50,12,I21.4,Circulatory diseases,38.0,58.0,NaN,NaN,NaN,NaN,347.0,78.0,173.0,NaN,100.0,5.13,NaN,5.30,8.0,19.0,156.0,0.45,2.0,2.0,NaN,1.20,3.5,6.0,8.0,NaN,NaN,10.90,341.0,30.0,88.0,NaN,11.5,NaN,0.1,138.0,66.0,12.7,14.4,131.0,367.0,4.5,NaN
4,2,142617,Entl. in ex.KH,m,51,1,T84.5,Injury & poisoning,15.0,65.0,19.0,0.04,16.0,2.39,NaN,88.0,71.0,0.09,87.0,5.05,17.0,7.00,3.0,NaN,135.0,0.42,1.0,NaN,0.01,1.86,3.7,21.0,NaN,NaN,0.84,8.46,327.0,27.0,81.0,0.71,11.3,7.15,NaN,136.0,35.2,19.2,17.6,159.0,NaN,5.2,NaN


In [37]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268781 entries, 0 to 268780
Data columns (total 51 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   patient_id           268781 non-null  int64  
 1   case_id              268781 non-null  int64  
 2   discharge_type       268781 non-null  object 
 3   sex                  268781 non-null  object 
 4   age                  268781 non-null  int64  
 5   length_of_stay_days  268781 non-null  int64  
 6   diagnosis            268781 non-null  object 
 7   diagnosis_category   268781 non-null  object 
 8   ALAT                 109650 non-null  float64
 9   AP                   80227 non-null   float64
 10  ASAT                 111518 non-null  float64
 11  BASm#n               70557 non-null   float64
 12  BIg                  80361 non-null   float64
 13  CA                   85867 non-null   float64
 14  CK                   84380 non-null   float64
 15  CR               

In [40]:
# Save output file
merged_data.to_csv(merged_data_path, index=False)

In [25]:
# Calculate missing percentage for each lab test
missing_percentage = merged_data.isnull().mean() * 100

# Add missing percentage to the summary statistics table
summary_stats = merged_data.describe().T
summary_stats['missing_percentage'] = missing_percentage

# Display the updated summary statistics table
summary_stats[4:].sort_values('missing_percentage')

,count,mean,std,min,25%,50%,75%,max,missing_percentage
KA,246422.0,4.029396,0.526045,1.20,3.70,4.00,4.300,34.10,8.318668
Leukn,241139.0,9.117285,6.532079,0.00,6.24,8.07,10.700,712.00,10.284209
Hbn,241136.0,125.103444,22.202181,0.00,112.00,127.00,141.000,298.00,10.285325
Eryn,241132.0,4.182007,0.753202,0.00,3.73,4.25,4.690,10.05,10.286813
Hkn,241131.0,0.366768,0.128104,0.00,0.33,0.37,0.410,42.00,10.287185
THZn,241128.0,235.961149,95.359648,0.00,180.00,225.00,278.000,3458.00,10.288302
MCHCn,241127.0,341.032398,13.299970,87.00,333.00,342.00,350.000,404.00,10.288674
MCHn,241126.0,30.015432,2.368148,5.00,29.00,30.00,31.000,51.00,10.289046
MCVn,241126.0,87.957151,5.922553,48.00,84.00,88.00,91.000,172.00,10.289046
RDWn,240903.0,14.004517,2.029728,0.10,12.80,13.40,14.600,43.20,10.372013
